Import Libraries

In [1]:
import os

In [2]:
home_dir = '../'
os.chdir(home_dir)
from ext import download_pubmed_articles, search_gene_id, download_gene_names
from norm import normalize_dis_genes
from relation_finder import find_sentences

"""
Required: A valid entrez registered email and entrez api-key.
More information: Entrez Programming Utilities (E-utilities)
"""  
from config import entrez_api_key, entrez_email

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
from Bio import Entrez
import xmltodict

Preprocessing <b>GAD</b><br>
<a href="https://geneticassociationdb.nih.gov/">GAD</a>

In [5]:
gad = pd.read_csv('Data/GADCDC_data.tsv', sep = '\t')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (1,7,8,10,11,21,23,24,26,28,35,37,39,42,45,47,51,52,53,54,55,57,58,59,60,61,62,63,64,65,66,67,68,77,78,83,84,86) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
gad.head(5)

,ID,ALTNAME,DISEASE,DIS_CLAS,CH_BAND,GENE,DNAPOST,P_VALUE,REFERENC,PUBMEDID,...,I_CHECK,UCSC,CHECKED,IPADDR,CNV,GEOLOC,MESHDIS,ACEVIEW,CTD,SNPEDIA
0,125158,||HLA-A||HLA-H||HLA-J||leukocyte antigen-A*010...,leukemia,CANCER,6p21.3,HLA-A,NaN,NaN,"Xiao, W. Y. et al. 2005",16120569.0,...,NaN,http://genome.ucsc.edu/cgi-bin/hgTracks?org=Hu...,NaN,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,NaN,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/HLA-A
1,125159,||HLA-A||HLA-H||HLA-J||leukocyte antigen-A*010...,leukemia,CANCER,6p21.3,HLA-A,NaN,NaN,"Zhou, L. X. et al. 2005",16143070.0,...,NaN,http://genome.ucsc.edu/cgi-bin/hgTracks?org=Hu...,NaN,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,NaN,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/HLA-A
2,125160,||HLA-A||HLA-H||HLA-J||leukocyte antigen-A*010...,alopecia areata,IMMUNE,6p21.3,HLA-A,NaN,NaN,"Xiao, F. L. et al. 2005",16185849.0,...,NaN,http://genome.ucsc.edu/cgi-bin/hgTracks?org=Hu...,NaN,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,Alopecia Areata,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/HLA-A
3,125155,||HLA-A||HLA-H||HLA-J||leukocyte antigen-A*010...,leukemia,CANCER,6p21.3,HLA-A,NaN,NaN,"Kang, L. et al. 2005",15793795.0,...,NaN,http://genome.ucsc.edu/cgi-bin/hgTracks?org=Hu...,NaN,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,NaN,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/HLA-A
4,125157,||HLA-A||HLA-H||HLA-J||leukocyte antigen-A*010...,melanoma,CANCER,6p21.3,HLA-A,NaN,NaN,"Spinola, H. et al. 2005",16101833.0,...,NaN,http://genome.ucsc.edu/cgi-bin/hgTracks?org=Hu...,NaN,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,NaN,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/HLA-A


In [7]:
print('No of entries in GAD:', gad.shape[0])

No of entries in GAD: 167132


In [8]:
gad = gad.dropna(subset = ['GENE', 'DISEASE', 'PUBMEDID'])
gad.index = gad.PUBMEDID.astype(int).astype(str)
gad = gad.drop(columns = ['PUBMEDID'])
gad.index.name = 'pmid'

In [9]:
# Create a list of all Pubmed IDs
pubmed_list = list(set([str(i) for i in gad.index.to_list()]))
# Download abstracts from pubmed IDs.
print('Set 1')
res1 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[:10000])
print('Set 2')
res2 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[10000:20000])
print('Set 3')
res3 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[20000:30000])
print('Set 4')
res4 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[30000:40000])
print('Set 5')
res5 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[40000:50000])
print('Set 6')
res6 = download_pubmed_articles(entrez_api_key, entrez_email, pubmed_list[50000:])

Set 1
No of unique pubmed ids: 10000
Decoding downloaded data...
Parsing to a dict...
9984 pubmed matches found out of 10000 pubmed articles.
Extracting abstracts and titles...
Done!
Set 2
No of unique pubmed ids: 10000
Decoding downloaded data...
Parsing to a dict...
9984 pubmed matches found out of 10000 pubmed articles.
Extracting abstracts and titles...
Done!
Set 3
No of unique pubmed ids: 10000
Decoding downloaded data...
Parsing to a dict...
9981 pubmed matches found out of 10000 pubmed articles.
Extracting abstracts and titles...
Done!
Set 4
No of unique pubmed ids: 10000
Decoding downloaded data...
Parsing to a dict...
9980 pubmed matches found out of 10000 pubmed articles.
Extracting abstracts and titles...
Done!
Set 5
No of unique pubmed ids: 10000
Decoding downloaded data...
Parsing to a dict...
9983 pubmed matches found out of 10000 pubmed articles.
Extracting abstracts and titles...
Done!
Set 6
No of unique pubmed ids: 4070
Decoding downloaded data...
Parsing to a dict...


In [10]:
res = res1 + res2 + res3 + res4 + res5 + res6
print(f'{len(res)} articles found out of {len(pubmed_list)}')

53980 articles found out of 54070


In [11]:
# Create DataFrame for downloaded pubmed articles.
pubmed_articles = pd.DataFrame(res).dropna()
pubmed_articles.index = pubmed_articles.pubmed
pubmed_articles.index.name = 'pmid'
pubmed_articles.abstract = pubmed_articles.apply(lambda x: x.title + ' ' + x.abstract, axis = 1)
pubmed_articles.head(5)

,pubmed,title,abstract
pmid,,,
19625333,19625333,Multiple sclerosis risk: interaction between h...,Multiple sclerosis risk: interaction between h...
17538887,17538887,HLA polymorphisms and cervical human Papilloma...,HLA polymorphisms and cervical human Papilloma...
19210878,19210878,Contribution of the R620W polymorphism of prot...,Contribution of the R620W polymorphism of prot...
19705255,19705255,Cardiovascular risk factors and outcome in pat...,Cardiovascular risk factors and outcome in pat...
18422024,18422024,Ala54Thr polymorphism of the fatty acid-bindin...,Ala54Thr polymorphism of the fatty acid-bindin...


In [12]:
# Merge GAD with pumed dataframe
df = gad.join(pubmed_articles).dropna(subset = ['abstract'])
df.head(5)

,ID,ALTNAME,DISEASE,DIS_CLAS,CH_BAND,GENE,DNAPOST,P_VALUE,REFERENC,ALLEL_AU,...,IPADDR,CNV,GEOLOC,MESHDIS,ACEVIEW,CTD,SNPEDIA,pubmed,title,abstract
pmid,,,,,,,,,,,,,,,,,,,,,
10022750,121451,||DRD2||D2DR||D2R||dopamine receptor D2||seven...,Cigarette Smoking,OTHER,11q23,DRD2,NaN,n,Singleton AB 1998,DRD2*A1,...,NaN,http://projects.tcag.ca/variation/search.asp?k...,Great Britain,Genetic Predisposition to Disease,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/DRD2,10022750,Lack of association between the dopamine D2 re...,Lack of association between the dopamine D2 re...
10023114,115029,||MGC26566||carboxycathepsin||DCP1||3.4.15.1||...,Parkinson's disease,NEUROLOGICAL,17q23,ACE,NaN,n,Mellick GD et al. 1999,NaN,...,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,Parkinson Disease,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/ACE,10023114,The ACE deletion polymorphism is not associate...,The ACE deletion polymorphism is not associate...
10023512,121385,||DRD2||D2DR||D2R||dopamine receptor D2||seven...,cocaine dependence,CHEMDEPENDENCY,11q23,DRD2,NaN,n,Gelernter J et al. 1999,NaN,...,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,Cocaine-Related Disorders|Genetic Predispositi...,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/DRD2,10023512,No association between D2 dopamine receptor (D...,No association between D2 dopamine receptor (D...
10023516,134812,||HTT||SERT||SLC6A4||SEROTONIN TRANSPORTER||5-...,anxiety-related traits,PSYCH,17q11.1-q12,SLC6A4,NaN,NaN,Katsuragi S et al. 1999,NaN,...,NaN,http://projects.tcag.ca/variation/search.asp?k...,Japan,Genetic Predisposition to Disease|Neurotic Dis...,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/SLC6A4,10023516,Association between serotonin transporter gene...,Association between serotonin transporter gene...
10023783,137170,||TP53||TRANSFORMATION-RELATED PROTEIN 53||tum...,cervical cancer,CANCER,17p13.1,TP53,NaN,n,Klaes R et al. 1999,NaN,...,NaN,http://projects.tcag.ca/variation/search.asp?k...,NaN,Papillomavirus Infections|Tumor Virus Infectio...,http://www.ncbi.nlm.nih.gov/IEB/Research/Acemb...,http://ctd.mdibl.org/detail.go?type=gene&db=GE...,http://www.snpedia.com/index.php/TP53,10023783,No evidence of p53 allele-specific predisposit...,No evidence of p53 allele-specific predisposit...


In [13]:
# Dictionary of genes
gene_dict = []
for i in range(len(df)):
    if isinstance(df.iloc[i].ALTNAME, str):
        aliases = [j for j in df.iloc[i].ALTNAME.split('||') if len(j) > 0]
    else:
        aliases = np.nan
    d = {'gene': df.iloc[i].GENE, 'aliases': aliases}
    if d['gene'].lower() not in [g['gene'].lower() for g in gene_dict]:
        gene_dict.append(d)

In [14]:
len(gene_dict)

14430

In [15]:
df['norm_dis'] = df.DISEASE.str.replace(' ', '_')

In [16]:
dis_dict = []
for i, row in df.iterrows():
    d = {'dis': row.DISEASE, 'aliases': row.norm_dis}
    if d['dis'].lower() not in [g['dis'].lower() for g in dis_dict]:
        dis_dict.append(d)

In [17]:
df['abstracts'] = normalize_dis_genes(gene_dict, dis_dict, df.GENE.to_list(), df.DISEASE.to_list(), df.abstract.to_list())
df = df[['GENE', 'norm_dis', 'abstract']]
df.columns = ['source', 'target', 'abstract']
rel_df = find_sentences(df)

144681 abstracts to be normalized...
144681 abstracts normalized...
Done!
144681 abstracts...
2431025 sentences found...
18869 sentences have a relation information.


In [18]:
rel_df.head(5)

,source,target,abstract,sentence,n_lines,match
pmid,,,,,,
10022750,DRD2,Cigarette_Smoking,Lack of association between the dopamine D2 re...,Lack of association between the dopamine D2 re...,1841,0
10022750,DRD2,Cigarette_Smoking,Lack of association between the dopamine D2 re...,The dopamine D2 receptor gene contains a TaqI...,1841,0
10022750,DRD2,Cigarette_Smoking,Lack of association between the dopamine D2 re...,It has been previously suggested that the les...,1841,0
10022750,DRD2,Cigarette_Smoking,Lack of association between the dopamine D2 re...,We genotyped a series of 104 smokers and 117 ...,1841,0
10022750,DRD2,Cigarette_Smoking,Lack of association between the dopamine D2 re...,A subset n = 87 of the smoking population als...,1841,0


In [19]:
rel_df = rel_df[['source', 'target', 'sentence', 'match']]

In [20]:
rel_df.to_csv('gad_norm_rel.csv')